In [26]:
# dependency and libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime

from config import weather_api_key

In [27]:
# random longs and lats
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

# zip ties each into tuples
lat_lngs = zip(lats, lngs)

# turn tuples into list (unzipping it)
coordinates = list(lat_lngs)

# create list of cities from coordinates
cities = []
for c in coordinates:
    city = citipy.nearest_city(c[0], c[1]).city_name

    if city not in cities:
        cities.append(city)
len(cities)

598

In [28]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"

# Make a 'Get' request for the city weather.
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")

City Weather found.


In [33]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list. -- LEARN ENUMERATE!
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# # Loop through all the cities in our list -- see above for ENUMERATE shortcut(?!)
# for i in range(len(cities)):

#     # Group cities in sets of 50 for logging purposes.
#     if (i % 50 == 0 and i >= 50):
#         set_count += 1
#         record_count = 1
#     # Create endpoint URL with each city.
#     city_url = url + "&q=" + cities[i]

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | nongstoin
Processing Record 2 of Set 1 | talnakh
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | bengkulu
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | new norfolk
Processing Record 7 of Set 1 | mar del plata
Processing Record 8 of Set 1 | tiznit
Processing Record 9 of Set 1 | cayenne
Processing Record 10 of Set 1 | thompson
Processing Record 11 of Set 1 | morondava
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | isangel
Processing Record 14 of Set 1 | atar
Processing Record 15 of Set 1 | east london
Processing Record 16 of Set 1 | mahebourg
Processing Record 17 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 18 of Set 1 | upernavik
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | alofi
Processing Record 21 of Set 1 | norman wells
Processing Record 22 of Set 1 | bluff
Processing